# Multi-Omics Cohort Building with CDA + ISB-CGC

This demo shows how to:
1. Build a multi-omics cohort with the [Cancer Data Aggregator (CDA)](https://cda.readthedocs.io/)
2. Overlapping sets with a Venn diagram (`venny4py`).  
3. Retrieve data from [ISB-CGC BigQuery](https://isb-cgc.appspot.com/)  
4. Visualizing results in **Holoviews** & **Panel** for an interactive dashboard.

## Prerequisites

### Mac Homebrew steps on command line:
```bash
brew install --cask google-cloud-sdk
```

### Python environment and other packages:
Then install and activate the `environment.yml` (Recommended):
```bash
conda create -f environment.yml
```

Alternatively, use pip:
```bash
pip install google-cloud-bigquery cdapython pandas numpy hvplot holoviews venny4py
```

## Imports and Config

In [ ]:
import pandas as pd
import numpy as np

from cdapython import tables, columns, column_values, fetch_rows

from google.cloud import bigquery

from venny4py.venny4py import venny4py

import holoviews as hv
import hvplot.pandas
import panel as pn

hv.extension('bokeh')
pn.extension()

print("Available CDA tables:", tables())

# columns(table=['subject'])
# column_values('data_type') # optional exploration

# Building a Multi-Omics Cohort in CDA

We will query four data types from the GDC, PDC, and IDC:
- Somatic mutations
- Gene expression
- Protein intensities
- Slide images

We then intersect these sets of Subject IDs to find the final cohort.


In [ ]:
# Queries
gdc_mutation = fetch_rows(
    table="subject",
    match_any=[
        "data_type = Aggregated Somatic Mutation",
        "data_type = Annotated Somatic Mutation"
    ],
    data_source=['GDC'],
    link_to_table='researchsubject'
)

gdc_expression = fetch_rows(
    table="subject",
    match_all=["data_type = Gene Expression Quantification"],
    data_source=['GDC'],
    link_to_table='researchsubject'
)

pdc_intensity = fetch_rows(
    table="subject",
    match_all=["data_type = Masked Intensities"],
    data_source=['PDC'],
    link_to_table='researchsubject'
)

idc_slide = fetch_rows(
    table="subject",
    match_all=["data_type = Slide Image"],
    data_source=['IDC'],
    link_to_table='researchsubject'
)

# Convert to sets and intersect
mut_set  = set(gdc_mutation['subject_id']) - {None}
expr_set = set(gdc_expression['subject_id']) - {None}
int_set  = set(pdc_intensity['subject_id'])  - {None}
slide_set= set(idc_slide['subject_id'])      - {None}

final_cohort = mut_set & expr_set & int_set & slide_set

print("Final cohort size:", len(final_cohort))

# Format them for BigQuery queries
formatted_identifiers = ",".join([f'"{x.split(".")[1]}"' for x in final_cohort if "." in x])


## Venn Diagram

We'll create a Venn diagram to show how many Subject IDs are in each set. This uses [venny4py](https://pypi.org/project/venny4py/).


In [ ]:
sets_dict = {
    'Mutation': mut_set,
    'Expression': expr_set,
    'Intensity': int_set,
    'Slide': slide_set
}

venn_result = venny4py(sets=sets_dict)
venn_result

# Retrieving Data from ISB-CGC BigQuery

We can now query the ISB-CGC BigQuery tables for:
1. **RNA expression** (ERBB2 example)
2. **Somatic mutations** (ERBB2 gene)
3. **Protein abundance** (ERBB2 again)

We'll then join them into a single table and explore. Make sure you have authenticated in a terminal:
```bash
gcloud auth application-default login
gcloud auth application-default set-quota-project YOUR_PROJECT_ID
```


In [ ]:
client = bigquery.Client('national-institute-of-health') # replace with your own gcp billing project

expression_query = f"""
SELECT
  case_barcode,
  project_short_name,
  gene_name,
  fpkm_uq_unstranded,
  sample_type_name
FROM `isb-cgc-bq.TCGA_versioned.RNAseq_hg38_gdc_r35`
WHERE case_barcode IN ({formatted_identifiers})
  AND gene_name = 'ERBB2'
  AND sample_type_name = 'Primary Tumor'
"""

expr_df = client.query(expression_query).to_dataframe()

mutation_query = f"""
SELECT
  case_barcode,
  project_short_name,
  SYMBOL,
  Variant_Classification
FROM `isb-cgc-bq.TCGA_versioned.masked_somatic_mutation_hg38_gdc_r36`
WHERE case_barcode IN ({formatted_identifiers})
  AND SYMBOL = 'ERBB2'
"""

mutation_df = client.query(mutation_query).to_dataframe()

abundance_query = f"""
SELECT
  meta.case_submitter_id,
  gene_symbol,
  quant.protein_abundance_log2ratio,
  aliquot.sample_type
FROM (
  SELECT * FROM `isb-cgc-bq.TCGA_versioned.quant_proteome_TCGA_breast_cancer_pdc_V2_10`
  UNION ALL
  SELECT * FROM `isb-cgc-bq.TCGA_versioned.quant_proteome_TCGA_ovarian_JHU_pdc_V2_10`
  UNION ALL
  SELECT * FROM `isb-cgc-bq.TCGA_versioned.quant_proteome_TCGA_ovarian_PNNL_pdc_V2_10`
) quant
JOIN `isb-cgc-bq.PDC_metadata_versioned.case_metadata_V2_10` meta
  ON quant.case_id = meta.case_id
JOIN `isb-cgc-bq.PDC_metadata_versioned.aliquot_to_case_mapping_V2_10` aliquot
  ON quant.aliquot_id = aliquot.aliquot_id
WHERE meta.case_submitter_id IN ({formatted_identifiers})
  AND gene_symbol = 'ERBB2'
"""

abundance_df = client.query(abundance_query).to_dataframe()

combined_query = f"""
SELECT
  expression.case_barcode,
  expression.project_short_name,
  expression.gene_name,
  fpkm_uq_unstranded,
  protein_abundance_log2ratio,
  Variant_Classification
FROM ({expression_query}) expression
JOIN ({abundance_query}) abundance
  ON expression.case_barcode = abundance.case_submitter_id
FULL OUTER JOIN ({mutation_query}) mutation
  ON expression.case_barcode = mutation.case_barcode
  AND expression.project_short_name = mutation.project_short_name
WHERE fpkm_uq_unstranded IS NOT NULL
"""

combined_df = client.query(combined_query).to_dataframe()

print("Combined dataframe shape:", combined_df.shape)
combined_df.head()


# Plotting with Holoviz + Bokeh

Let's do a scatter plot of log(FPKM) vs. protein abundance, and a box plot grouping by mutation type.


In [ ]:
# Create a log-transformed column
combined_df['log_fpkm_uq'] = np.log(
    combined_df['fpkm_uq_unstranded'].replace(0, np.nan)
)

# # Scatter plot
# scatter_plot = combined_df.hvplot.scatter(
#     x='log_fpkm_uq',
#     y='protein_abundance_log2ratio',
#     title='ERBB2: RNA vs. Protein',
#     xlabel='log(FPKM_UQ)',
#     ylabel='Protein Abundance (log2ratio)',
#     height=400,
#     width=400
# )

# Box plot: separate Missense vs Wild Type
combined_df['Mutation_Status'] = np.where(
    combined_df['Variant_Classification'] == 'Missense_Mutation',
    'Missense',
    'Wild Type'
)

box_plot = combined_df.hvplot.box(
    y='log_fpkm_uq',
    by='Mutation_Status',
    title='ERBB2 Expression by Mutation Status',
    xlabel='Mutation Status',
    ylabel='log(FPKM_UQ)',
    height=400,
    responsive=True
)

fpkm_slider = pn.widgets.FloatSlider(
    name='Min log(FPKM_UQ)',
    start=0.0,
    end=float(combined_df['log_fpkm_uq'].max() or 10),
    step=0.2,
    value=0.0
)

# adding a threshold widget just to demonstrate interactivity with panel
@pn.depends(fpkm_slider.param.value)
def filtered_scatter(fpkm_val):
    subset = combined_df[combined_df['log_fpkm_uq'] >= fpkm_val]
    return subset.hvplot.scatter(
        x='log_fpkm_uq',
        y='protein_abundance_log2ratio',
        c='Mutation_Status',
        title=f"Filtered RNA vs Protein (min log(FPKM)={fpkm_val:.1f})",
        frame_height=400, responsive=True,
    ) * hv.VLine(fpkm_val)

demo_app = pn.Column(fpkm_slider,
    pn.Row(filtered_scatter, box_plot),
)

demo_app